In [5]:
from bs4 import BeautifulSoup as soup
import requests as req
import pandas as pd
req_url = req.get('https://www.imdb.com/search/title?title_type=feature&num_votes=25000,&genres=action&start=0&ref_=adv_nxt')
Content = req_url.content

page = soup(Content,'html.parser')

#print(soup.prettify(movies_container[0]))

movies_container = page.findAll('div',{'class':'lister-item-content'})
print(len(movies_container))

50


In [6]:
#print(soup.prettify(movies_container[0]))

In [7]:
DataSet = []
number_of_moives = len(movies_container)

i = 0
while(i<number_of_moives):
    movies = {}
    m = movies_container[i]
    try:
        movies['Name'] = m.find(class_='lister-item-header').get_text(strip=True)
    except AttributeError:
        moves['Name'] = None
    try:
        movies['Certificate'] = m.find(class_='certificate').get_text(strip=True)
    except AttributeError:
        movies['Certificate'] = None
    try:
        movies['Runtime'] = m.find(class_='runtime').get_text(strip=True)
    except AttributeError:
        movies['Runtime'] = None
    try:
        movies['Genre'] = m.find(class_='genre').get_text(strip=True)
    except AttributeError:
        movies['Genre'] = None
    try:
        movies['Rating'] = m.find(class_='inline-block ratings-imdb-rating').get_text(strip=True)
    except AttributeError:
        movies['Rating'] = None
    try:
        movies['Metascore'] = m.find(class_='inline-block ratings-metascore').get_text(strip=True)
    except AttributeError:
        movies['Metascore'] = None
    try:
        votes_and_gross = m.find(class_='sort-num_votes-visible').get_text(strip=True)
        v= votes_and_gross.split('|')
        n = v[0].split(':')#
        movies['Votes'] = n[1]
    except AttributeError:
        movies['Votes'] = None
    i = i + 1
    DataSet.append(movies)

dataframe = pd.DataFrame(DataSet)
dataframe['Name'] = dataframe['Name'].astype(str).str[2:]
dataframe['Metascore'] = dataframe['Metascore'].astype(str).str[0:2]

In [8]:
col_names = ["Name","Certificate",'Runtime',"Genre","Rating",'Metascore',"Votes"]
dataframe = dataframe.reindex(columns=col_names)
dataframe.to_csv("Output.csv")
dataframe.head()

,Name,Certificate,Runtime,Genre,Rating,Metascore,Votes
0,Alita: Battle Angel(2019),PG-13,122 min,"Action, Adventure, Romance",7.6,54,"62,445"
1,Spider-Man: Into the Spider-Verse(2018),PG,117 min,"Animation, Action, Adventure",8.6,87,"128,726"
2,Black Panther(2018),PG-13,134 min,"Action, Adventure, Sci-Fi",7.3,88,"470,688"
3,Polar(2019),None,118 min,"Action, Crime, Drama",6.3,19,"43,595"
4,Aquaman(2018),PG-13,143 min,"Action, Adventure, Fantasy",7.3,55,"175,608"
